In [3]:
#입력한 문단을 .을 기준으로 구분(개별문장 구분)후 감성분석한다.
#분석결과중 neutral을 제외하고 가장 높은 점수를 얻은 대표감성을 산출하고
#최종결과로 문장과 대표감성만을 추출하여 리턴하는 함수

#관련: accu-critique


from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline

# from essayai.ai_plot.model import BertForMultiLabelClassification
# from essayai.ai_plot.multilabel_pipeline import MultiLabelPipeline

# from essayai.ai_emotion.model import BertForMultiLabelClassification
# from essayai.ai_emotion.multilabel_pipeline import MultiLabelPipeline
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np

import operator

tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-ekman")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-ekman")

goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kimkwangil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#데이터 전처리 
def cleaning(datas):

    fin_datas = []

    for data in datas:
        # 영문자 이외 문자는 공백으로 변환
        only_english = re.sub('[^a-zA-Z]', ' ', data)
    
        # 데이터를 리스트에 추가 
        fin_datas.append(only_english)

    return fin_datas

In [5]:
def get_ai_emotion(input_text) :
    
    joy = 0
    anger = 0
    sadness = 0
    surprise = 0
    fear = 0
    neutral = 0
    
    list_max = []
    
    try : 
        
        re_text = input_text.split(".")
        texts = cleaning(re_text)
        print("전처리 결과", texts)
        re_emot =  goemotions(texts)  ####### 감정 계산하는 부분 
        
        
        df = pd.DataFrame(re_emot)

        #결과물을 다시 감정유형별 비율로 계산하여 새로운 데이터프레임을 만들자!
        # result_emotion = df['labels'].value_counts(normalize=True, sort=True, ascending=False, dropna=True) #문장 전체에서 각 값의 상대적 비율을 게산
        result_emotion = df['labels'].value_counts(normalize=True) #문장 전체에서 각 값의 상대적 비율을 게산
        #print("index", result_emotion.index[0][0])


        result_emotion_list = result_emotion.tolist()
        

        
        idx=0
        
        for  idx  in range(len(result_emotion)) : 
                
                
                if  'joy' in result_emotion.index[idx][0]: 
                    joy += round(float(result_emotion_list[idx]),2)
                    print("joy >>>>>>>>>>>>>>>>:", joy)
                    
                    
                elif 'anger' in result_emotion.index[idx][0]:
                    anger += round(float(result_emotion_list[idx]),2)
                    print("anger >>>>>>>>>>>>>>>>:", anger)
                    
                    

                elif 'sadness' in  result_emotion.index[idx][0]:
                    sadness += round(float(result_emotion_list[idx]),2)
                    print("sadness >>>>>>>>>>>>>>>>:", sadness)

                    

                elif 'surprise' in result_emotion.index[idx][0]:
                    surprise += round(float(result_emotion_list[idx]),2)
                    print("surprise >>>>>>>>>>>>>>>>:", surprise)

                    

                elif 'fear' in result_emotion.index[idx][0]:
                    fear += round(float(result_emotion_list[idx]),2)
                    print("fear >>>>>>>>>>>>>>>>:", fear)

                    

                elif 'neutral' in result_emotion.index[idx][0]:
                    neutral += round(float(result_emotion_list[idx]),2)
                    print("neutral >>>>>>>>>>>>>>>>:", neutral)

                    

                idx = idx + 1
        
        
            
        list_max = [joy,anger,sadness,surprise,fear,neutral]
    
        # print("max:",list_max.index(max(list_max))) 
        
        
        
    except Exception as e : 
        print ("------------------------->error:",e)
    
    try : 
       max_value = list_max.index(max(list_max))+1 
    
    except Exception as e :
        max_value = 2 
        print("Exception_-------->",e)
            
    #딕셔너리로 결과 추출
    data = {
        "joy" : joy,
        "anger" : anger,
        "sadness" : sadness,
        "surprise" : surprise,
        "fear" : fear,
        #"neutral" : neutral,
        #"max": max_value
    }
    
    #가장 높은 값 추출
    data_values = sorted(data.items(), key=operator.itemgetter(1))
    print("dtype: ", type(data_values)) #list
    print("data_vlaues: ", data_values)
    return re_text, data_values[4]  #리스트의 마지막 값이 가장 높은 점수를 받은 대표 감성
    

text = """ Tom was in love with Cyndi.She loved Tom too because 
he was so nice and kind."""
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",get_ai_emotion(text))


In [6]:
text = """ Tom was in love with Cyndi. He really loved her dearly. She loved Tom too because 
he was so nice and kind. They would walk in the peaceful park gazing at the bright pink cherry 
blossom that colored the crisp spring air. Then they got into a big fight. Tom began to hate 
Cyndi's guts. Cyndi cheated on Tom and she began to ignore Tom. He eventually became furious. 
Back at their house, Tom began to slap Cyndi and she was crying miserably. However, the sun 
shined upon them when they won the lotto. Both of them were ecstatic and overjoyed with happiness. 
They where so happy that they jumped around in celebration. Nonetheless, the happiness never
lasted too long. Cyndi began to cheat on Tom again and they went bankrupt very quickly. Fights 
and quarrels continued in the old torn smelly apartment. It really sucked for both of them. 
Extreme sadness came over them. But then Cyndi bought another lotto and she won again. 
This time, it was even more money and they were extremely excited. To celebrate the second winning,
then went to a casino and began gambling. They were overly excited because they won money at the 
fancy casino too. Nevertheless, Tom had too much drink and died. Cyndi was so sad. Didn't know 
what to do and weeped in despair. """
#print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",get_ai_emotion(text))


In [7]:
def splitParagraphIntoSentences(paragraph):
  import re
  sentenceEnders = re.compile(r"""
      # Split sentences on whitespace between them.
      (?:               # Group for two positive lookbehinds.
        (?<=[.!?])      # Either an end of sentence punct,
      | (?<=[.!?]['"])  # or end of sentence punct and quote.
      )                 # End group of two positive lookbehinds.
      (?<!  Mr\.   )    # Don't end sentence on "Mr."
      (?<!  Mrs\.  )    # Don't end sentence on "Mrs."
      (?<!  Jr\.   )    # Don't end sentence on "Jr."
      (?<!  Dr\.   )    # Don't end sentence on "Dr."
      (?<!  Prof\. )    # Don't end sentence on "Prof."
      (?<!  Sr\.   )    # Don't end sentence on "Sr."."
    \s+               # Split on whitespace between sentences.
    """, 
    re.IGNORECASE | re.VERBOSE)
  sentenceList = sentenceEnders.split(paragraph)
  return sentenceList

In [8]:
text_split_sentence = splitParagraphIntoSentences(text)
text_split_sentence

[' Tom was in love with Cyndi.',
 'He really loved her dearly.',
 'She loved Tom too because \nhe was so nice and kind.',
 'They would walk in the peaceful park gazing at the bright pink cherry \nblossom that colored the crisp spring air.',
 'Then they got into a big fight.',
 "Tom began to hate \nCyndi's guts.",
 'Cyndi cheated on Tom and she began to ignore Tom.',
 'He eventually became furious.',
 'Back at their house, Tom began to slap Cyndi and she was crying miserably.',
 'However, the sun \nshined upon them when they won the lotto.',
 'Both of them were ecstatic and overjoyed with happiness.',
 'They where so happy that they jumped around in celebration.',
 'Nonetheless, the happiness never\nlasted too long.',
 'Cyndi began to cheat on Tom again and they went bankrupt very quickly.',
 'Fights \nand quarrels continued in the old torn smelly apartment.',
 'It really sucked for both of them.',
 'Extreme sadness came over them.',
 'But then Cyndi bought another lotto and she won aga

In [9]:
result_all_emo_sent = []
for line_ in text_split_sentence:
    emo_analy_sentence = get_ai_emotion(line_)
    result_all_emo_sent.append(emo_analy_sentence)

전처리 결과 [' Tom was in love with Cyndi', '']
joy >>>>>>>>>>>>>>>>: 0.5
neutral >>>>>>>>>>>>>>>>: 0.5
dtype:  <class 'list'>
data_vlaues:  [('anger', 0), ('sadness', 0), ('surprise', 0), ('fear', 0), ('joy', 0.5)]
전처리 결과 ['He really loved her dearly', '']
joy >>>>>>>>>>>>>>>>: 0.5
neutral >>>>>>>>>>>>>>>>: 0.5
dtype:  <class 'list'>
data_vlaues:  [('anger', 0), ('sadness', 0), ('surprise', 0), ('fear', 0), ('joy', 0.5)]
전처리 결과 ['She loved Tom too because  he was so nice and kind', '']
joy >>>>>>>>>>>>>>>>: 0.5
neutral >>>>>>>>>>>>>>>>: 0.5
dtype:  <class 'list'>
data_vlaues:  [('anger', 0), ('sadness', 0), ('surprise', 0), ('fear', 0), ('joy', 0.5)]
전처리 결과 ['They would walk in the peaceful park gazing at the bright pink cherry  blossom that colored the crisp spring air', '']
joy >>>>>>>>>>>>>>>>: 0.5
neutral >>>>>>>>>>>>>>>>: 0.5
dtype:  <class 'list'>
data_vlaues:  [('anger', 0), ('sadness', 0), ('surprise', 0), ('fear', 0), ('joy', 0.5)]
전처리 결과 ['Then they got into a big fight', '']
neu

In [10]:
result_all_emo_sent[:3]

[([' Tom was in love with Cyndi', ''], ('joy', 0.5)),
 (['He really loved her dearly', ''], ('joy', 0.5)),
 (['She loved Tom too because \nhe was so nice and kind', ''], ('joy', 0.5))]

In [11]:
len(result_all_emo_sent)

25

In [12]:
[x[0] for x in result_all_emo_sent][0][0]

' Tom was in love with Cyndi'

In [13]:
[x[0] for x in result_all_emo_sent][1][0]

'He really loved her dearly'

In [14]:
[x[1] for x in result_all_emo_sent][0][0]

'joy'

In [15]:
[x[1] for x in result_all_emo_sent][1][0]

'joy'

In [16]:
#문장을 하나씩 분석하여 감성정보를 분석 후 각 값을 매칭시킴
#결과값은 딕셔너리로 {문장 : 감성분석결과}
get_sentences = {}
cnt = 0
for cnt in range(0, len(result_all_emo_sent)):
    s_ = [x[0] for x in result_all_emo_sent][cnt][0]
    e_ = [x[1] for x in result_all_emo_sent][cnt][0]
    get_sentences[s_] = e_
    cnt += 1

In [17]:
#결과 추출 : 이제 자바스크립트로 웹페이지 감성정보를 추출하여 밑줄긋도 분석결과 정보 제공하면 됨
get_sentences

{' Tom was in love with Cyndi': 'joy',
 'He really loved her dearly': 'joy',
 'She loved Tom too because \nhe was so nice and kind': 'joy',
 'They would walk in the peaceful park gazing at the bright pink cherry \nblossom that colored the crisp spring air': 'joy',
 'Then they got into a big fight': 'fear',
 "Tom began to hate \nCyndi's guts": 'anger',
 'Cyndi cheated on Tom and she began to ignore Tom': 'fear',
 'He eventually became furious': 'fear',
 'Back at their house, Tom began to slap Cyndi and she was crying miserably': 'fear',
 'However, the sun \nshined upon them when they won the lotto': 'fear',
 'Both of them were ecstatic and overjoyed with happiness': 'joy',
 'They where so happy that they jumped around in celebration': 'joy',
 'Nonetheless, the happiness never\nlasted too long': 'fear',
 'Cyndi began to cheat on Tom again and they went bankrupt very quickly': 'fear',
 'Fights \nand quarrels continued in the old torn smelly apartment': 'fear',
 'It really sucked for both 